# What is a Jupyter notebook?

A Jupyter notebook is a way for you write and execute Python in your web browser.  It's easy to interleave text, code, and the outputs of code.  This makes notebooks useful as a kind of interactive textbook.

## Getting started

The document that you are reading is not a static web page, but an interactive environment called a "Jupyter notebook" that lets you write and execute code.

For example, below is a code cell with a short Python script that computes a value, stores it in a variable and prints the result.

In [ ]:
seconds_in_a_day = 24 * 60 * 60
seconds_in_a_day

To execute the code in the above cell, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut 'Shift+Enter'. To edit the code, just click the cell and start editing.

Variables that you define in one cell can later be used in other cells:

In [ ]:
seconds_in_a_week = 7 * seconds_in_a_day
seconds_in_a_week

Jupyter notebooks allow you to combine executable code and rich text in a single document, along with images, HTML and more. To find out more about the Jupyter project, see [jupyter.org](https://www.jupyter.org).

## GOV.UK as a network

This section visualises part of GOV.UK as a network of pages, linked to each other.

Most of the Python code has been hidden from view, because we're not teaching Python in this workshop.  But if you are interested in seeing the code, then explore it on [GitHub](https://github.com/alphagov/govuk-data-science-workshop).

Run the two cells below (shift + enter).  Try changing the search term, checkbox, radio buttons and community filter and then run the second cell again to see how the network visualisation changes.

In [ ]:
import os

DIR_SRC_UTILS = os.getenv("DIR_SRC_UTILS")
execfile(DIR_SRC_UTILS + "/setup.py")

In [ ]:
community_graph(
    edges,
    search_term=search_term.value,
    display_centrality=display_centrality.value,
    community_algorithm=algorithm.value,
    community_filter=community_filter.value,
)

## Ranking search results

Another way that we can use node centrality is to rank search results.

The cell below obtains the GOV.UK pages that have the highest 'degree' centrality.  Are these good enough to use as search results?

In [ ]:
top_results_by_degree = top_nodes(
    edges,
    search_term=search_term.value,
    centrality_algorithm="degree",
    number_of_results=10,
)
top_results_by_degree

'Degree' centrality isn't the only way to measure the importance of a node.  Try using Google's 'pagerank' algorithm by changing the value of the `centrality_algorithm` in the code below.  How are these results different from the ones obtained by 'degree' centrality?

In [ ]:
top_results_by_pagerank = top_nodes(
    edges,
    search_term=search_term.value,
    centrality_algorithm="pagerank",
    number_of_results=10,
)
top_results_by_pagerank

## Jaccard: how similar are two sets of search results?

We have compared two different sets of search results by eye.  Is there some kind of similarity score we can calculate instead?  A number that is high when the results are similar, and low when they are dissimilar?

### Definition

Jaccard similarity is the amount of overlap between two sets of items.  It is the number of items in common, divided by the number of items that are different.

$$\mathrm{Jaccard} = {\mathrm{number \space of \space items \space in \space common}\over\mathrm{number \space of \space items \space not \space in \space common}}$$

### Code

You can find the mathematical definition of Jaccard similarity on [Wikipedia](https://en.wikipedia.org/wiki/Jaccard_index) and [StackOverflow](https://stackoverflow.com/a/47016862/937932).  Another way to understand it is to implement it in code, as below.

In [ ]:
def jaccard(set_1, set_2):

    # Ensure that set_1 and set_2 are python set objects, not merely list objects
    set_1 = set(set_1)
    set_2 = set(set_2)

    intersection = set_1 & set_2  # One of each thing that appears in both sets
    union = set_1 | set_2  # One of each thing that appears in either set

    jaccard = float(len(intersection) / len(union))

    # Calculate the Jaccard index
    print(
        "\nIntersection: " + str(len(intersection)) + "\n  " + "\n  ".join(intersection)
    )
    print("\nUnion: " + str(len(union)) + "\n  " + "\n  ".join(union))
    print("\nJaccard:\n  " + str(jaccard))

Now use the jaccard function to compare two sets of things.  Instead of comparing sets of URLs, we use single letters, to make it easier to understand what the code is doing.

In [ ]:
set_1 = set(["c", "a", "t"])  # A set of things
set_2 = set(["m", "a", "t"])  # Another set of things

jaccard(set_1, set_2)

### Checking the code

Check that the result 2.0 is correct, by doing the calculation by hand.

* **Intersection**: letters 'a' and 't' are in both sets.  So the intersection is 2 (two letters).
* **Union**: there are four letters in total 'c', 'a', 't' and 'm'.  Each letter is counted only once, even though letters 'a' and 't' appear twice each.

The Jaccard index is the size of the intersection, 2, divided by the size of the union, 4.  The result is 0.5, the same as the code calculated.

Try calculating the Jaccard index of different sets.  For example, change `['c', 'a', 't']` to `'['c', 'a', 't', 's']` or `['c', 'a', 'r']`.  Can you get a score above 1?  What about a negative score?

### Comparing the results of 'degree' and 'pagerank' centrality

Now we can create a score for how similar the 'degree' centrality search results are to the 'pagerank' centrality ones.

First, a reminder of what the results were.  Roughly what score would you expect?

In [ ]:
top_results_by_degree

In [ ]:
top_results_by_pagerank

Now compare the results with Jaccard.

In [ ]:
jaccard(top_results_by_degree.url, top_results_by_pagerank.url)

Six of the top 10 results of each method are the same -- not bad, considering that the 'degree' centrality is much more straightforward to calculate than the pagerank.

### Compare with actual GOV.UK search

The real GOV.UK search function uses [ElasticSearch](https://www.elastic.co/elastic-stack/) with some modifications, notably a [Learn to Rank algorithm](https://docs.publishing.service.gov.uk/apps/search-api/arch/adr-010-learn-to-rank.html).  How do its search results compare with the ones obtained with 'degree' and 'pagerank' centrality?

In [ ]:
govuk_results = govuk_search(search_term="childcare", number_of_results=10)
govuk_results

Jaccard can only compare pairs of results, so we have to use it twice: between 'govuk' and 'degree', and between 'govuk' and 'pagerank'.

In [ ]:
jaccard(govuk_results.url, top_results_by_degree.url)

In [ ]:
jaccard(govuk_results.url, top_results_by_pagerank.url)

### Comparing search with itself

The GOV.UK search algorithm seems much more sophisticated than the 'degree' and 'pagerank' ones we built ourselves.  
Let's test it against itself by trying to trick it.  How does it cope with plurals, or misspellings? Do they return similar results?

In [ ]:
jaccard(govuk_search("benifit", 10).url, govuk_search("benefits", 10).url)

## Review

That's the end of this exercise.  In it, you learned:

- What GOV.UK looks like as a network
- The effect of different clustering algorithms
- Two algorithms to calculate the importance of a node
- How to compare sets of search results with the Jaccard score

Hopefully next time you encounter search results you will think about:

- How the results might have been obtained
- What makes results relevant
- Whether a single algorithm can ever be the best at everything